In [2]:
import spacy
import pdfplumber
import re
import os
import pandas as pd
import PyPDF2


In [3]:


title_keywords = ['recipe', 'title', 'name']
ingredient_keywords = ['ingredients', 'what you need', 'you will need', 'needs', 'ingredient list', 'required', 'items needed']
instruction_keywords = ['instructions', 'method', 'directions', 'how to', 'preparations']

recipes = []

pdf_folder = "C:\\Users\\oyku_\\Desktop\\Recipe Collector\\Recipe-Collector\\pdf_collectors\\test"
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        with open(os.path.join(pdf_folder, filename), 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            for page in reader.pages:
                text += page.extract_text() + '\n'
            
            lines = text.split('\n')
            title, ingredients, instructions = None, [], []
            in_ingredients = False
            in_instructions = False
            
            for line in lines:
                if any(keyword.lower() in line.lower() for keyword in title_keywords):
                    if title is not None:  # Save the previous recipe if it exists
                        recipes.append({'Title': title, 'Ingredients': ', '.join(ingredients), 'Instructions': '\n'.join(instructions)})
                    title = line.strip()
                    ingredients = []
                    instructions = []
                    in_ingredients = False
                    in_instructions = False
                
                # Check for ingredients 
                if any(keyword.lower() in line.lower() for keyword in ingredient_keywords):
                    in_ingredients = True
                    continue
                
                # Check for instructions 
                if any(keyword.lower() in line.lower() for keyword in instruction_keywords):
                    in_instructions = True
                    in_ingredients = False
                    continue

                # Collect ingredients
                if in_ingredients:
                    if line.strip():  
                        ingredients.append(line.strip())
                
                # Collect instructions
                if in_instructions:
                    if line.strip():  
                        instructions.append(line.strip())
            
            if ingredients and title:  # Only save if both title and ingredients are present
                recipes.append({'Title': title, 'Ingredients': ', '.join(ingredients), 'Instructions': '\n'.join(instructions)})
            else:
                print(f"No ingredients found in {filename}. Skipping.")

# Save to CSV
df = pd.DataFrame(recipes)
df.to_csv('pdf_try_test_1.csv', index=False)


No ingredients found in 03.-Dessert-Lovers-Foodbook-author-Myfoodbook.pdf. Skipping.
No ingredients found in 03.-Enjoy-Yeast-Breads-Both-Plain-and-Fancy-author-Elna-Miller.pdf. Skipping.
No ingredients found in 09. Pisco Sour Autor JMA Design %26 Community.pdf. Skipping.
No ingredients found in 9. Tea and Health %28Article%29 Author The Tea Association of The USA.pdf. Skipping.


In [8]:
import os
import pandas as pd
import PyPDF2

# Define your keywords
title_keywords = ['recipe', 'title', 'name']
ingredient_keywords = ['ingredients', 'what you need', 'you will need', 'needs', 'ingredient list', 'required', 'items needed']
instruction_keywords = ['instructions', 'method', 'directions', 'how to', 'preparations']

# Prepare a list to store recipes and a list for tossed PDFs
recipes = []
tossed_pdfs = []

# Loop through each PDF in the folder
pdf_folder = "C:\\Users\\oyku_\\Desktop\\Recipe Collector\\Recipe-Collector\\pdf_collectors\\test"
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        with open(os.path.join(pdf_folder, filename), 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            for page in reader.pages:
                text += page.extract_text() + '\n'
            
            # Process the text to extract title, ingredients, and instructions
            lines = text.split('\n')
            title, ingredients, instructions = None, [], []
            in_ingredients = False
            in_instructions = False
            
            for line in lines:
                line = line.strip()  # Clean up whitespace

                # Check for title
                if any(keyword.lower() in line.lower() for keyword in title_keywords):
                    # Save previous recipe if it exists
                    if title is not None and ingredients: 
                        recipes.append({
                            'Title': title,
                            'Ingredients': ', '.join(ingredients),
                            'Instructions': '\n'.join(instructions)
                        })
                    
                    title = line  # Update the title
                    ingredients = []  # Reset ingredients
                    instructions = []  # Reset instructions
                    in_ingredients = False
                    in_instructions = False
                    print(f"Found title: {title}")  # Debugging statement
                    continue  # Skip to the next line

                # Check for ingredients section
                if any(keyword.lower() in line.lower() for keyword in ingredient_keywords):
                    in_ingredients = True
                    in_instructions = False  # Ensure we are not in instructions
                    print(f"Found ingredients section starting at: {line}")  # Debugging statement
                    continue
                
                # Check for instructions section
                if any(keyword.lower() in line.lower() for keyword in instruction_keywords):
                    in_instructions = True
                    in_ingredients = False  # Ensure we are not in ingredients
                    print(f"Found instructions section starting at: {line}")  # Debugging statement
                    continue

                # Collect ingredients
                if in_ingredients:
                    if line:  # Ensure it's not empty
                        ingredients.append(line)
                        print(f"Added to ingredients: {line}")  # Debugging statement
                
                # Collect instructions
                if in_instructions:
                    if line:  # Ensure it's not empty
                        instructions.append(line)
                        print(f"Added to instructions: {line}")  # Debugging statement
            
            # After finishing the document, check if we collected a recipe
            if title and ingredients:  # Only save if both title and ingredients are present
                recipes.append({
                    'Title': title, 
                    'Ingredients': ', '.join(ingredients), 
                    'Instructions': '\n'.join(instructions)
                })
            else:
                tossed_pdfs.append(filename)  # Log the tossed PDF
                print(f"Tossed PDF: {filename} - Missing title or ingredients")  # Debugging statement

# Save to CSV
df = pd.DataFrame(recipes)
df.to_csv('pdf_try_test_2.csv', index=False)

# Output tossed PDFs
if tossed_pdfs:
    print("The following PDFs were tossed due to missing ingredient sections:")
    for tossed in tossed_pdfs:
        print(tossed)
else:
    print("No PDFs were tossed.")


Found title: PART II. TRY UTAH’S TESTED RECIPES
Found title: “ S t a f f  o f  L i f e ”  B r e a d ....................................................1 9R e c i p e  f o r  E n r i c h e d  W h i t e  B r e a d ..........................................2 0R e c i p e  f o r  W h o l e  W h e a t  B r e a d ............................................2 1F a n c y  Y e a s t  B r e a d s .....................................................2 2M a k e  S o m e  B a t t e r  B r e a d .................................................2 3Foundation Recipe for Sweet Yeast Dough ...................................2 5
Found instructions section starting at: You will find, in the pages which follow, many changes from the bread-making methods
Found title: and recipes prepared for use at sea level.
Found title: Read the general instructions before you try any of these recipes. They will help you to
Found instructions section starting at: understand why these methods are recommended. Follow the d

In [ ]:
import pdfplumber
import pandas as pd
import os

# Define keywords
ingredient_keywords = [
    'ingredients', 'what you need', 'you will need', 
    'needs', 'ingredient list', 'required', 'items needed'
]
instruction_keywords = [
    'instructions', 'method', 'directions', 'how to', 'preparations'
]

# Function to find keywords
def find_section(text, keywords):
    return any(keyword in text.lower() for keyword in keywords)

# Prepare the DataFrame
data = []

# Folder containing PDFs
folder_path = "C:\\Users\\oyku_\\Desktop\\Recipe Collector\\Recipe-Collector\\pdf_collectors\\test"

# Iterate through each PDF
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        with pdfplumber.open(os.path.join(folder_path, filename)) as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text() + '\n'
            
            lines = text.split('\n')
            title = []
            ingredients = []
            instructions = []
            in_ingredients = False
            in_instructions = False

            # Debugging: Print the lines extracted from the PDF
            print(f"Processing {filename}:")
            for line in lines:
                line = line.strip()

                # Check if we found any keywords
                if find_section(line, ingredient_keywords):
                    print(f"Found ingredients keyword in line: {line}")
                if find_section(line, instruction_keywords):
                    print(f"Found instructions keyword in line: {line}")
                
                # Capture title lines (up to 3 lines before ingredients)
                if not in_ingredients and not in_instructions:
                    if len(title) < 3:  # Limit to 3 lines for title
                        title.append(line)
                    continue  # Continue until we find the ingredients section
                
                # Check for ingredients section
                if find_section(line, ingredient_keywords):
                    in_ingredients = True
                    print("Started capturing ingredients.")
                    continue  # Skip to the next line

                # Check for instructions section
                if in_ingredients and find_section(line, instruction_keywords):
                    in_ingredients = False
                    in_instructions = True
                    print("Stopped capturing ingredients, started capturing instructions.")
                    continue  # Skip to the next line

                # Capture ingredients
                if in_ingredients and line:
                    ingredients.append(line)

                # Capture instructions
                if in_instructions and line:
                    instructions.append(line)

            # Combine title lines into a single string
            title_str = ' '.join(title).strip()

            # Check if ingredients were found
            if ingredients:
                # Join ingredients and instructions into strings
                data.append({
                    'Title': title_str,
                    'Ingredients': ', '.join(ingredients),
                    'Instructions': '\n'.join(instructions)
                })
                print(f"Added recipe from {filename}: {title_str}")
            else:
                print(f"Skipped {filename}: No ingredients found.")

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('output_recipes.csv', index=False)

print("Processing complete. Check output_recipes.csv for results.")


In [35]:
import os
import re
import pdfplumber
import pandas as pd

def extract_recipes_from_pdf(pdf_path):
    recipes = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            print("Extracted Text:\n", text)  # Debugging line
            
            if text:
                # Extracting title (first line)
                title_pattern = r'^(.*)$'
                title_match = re.search(title_pattern, text, re.MULTILINE)
                title = title_match.group(1) if title_match else None

                # Extracting ingredients (all lines until the "Instructions" section)
                ingredients_pattern = r'(?i)ingredients\s*:\s*(.*?)(?=\n\s*instructions\s*:|$)'
                ingredients_match = re.search(ingredients_pattern, text, re.DOTALL)
                ingredients = ingredients_match.group(1).strip() if ingredients_match else None

                # Extracting instructions (all lines after the "Instructions" section)
                instructions_pattern = r'(?i)instructions\s*:\s*(.*?)(?=\n\s*$|$)'
                instructions_match = re.search(instructions_pattern, text, re.DOTALL)
                instructions = instructions_match.group(1).strip() if instructions_match else None

                # Print the extracted parts for debugging
                print("Title Match:", title)  # Debugging line
                print("Ingredients Match:", ingredients)  # Debugging line
                print("Instructions Match:", instructions)  # Debugging line

                # Append only if all parts are present
                if title and ingredients and instructions:
                    recipes.append({
                        'Title': title,
                        'Ingredients': ingredients,
                        'Instructions': instructions
                    })
                else:
                    print("Missing one or more parts for this recipe.")  # Log if any part is missing

    print("Recipes found:", recipes)  # Add this line before returning
    return recipes

def process_pdf_folder(folder_path):
    all_recipes = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            recipes = extract_recipes_from_pdf(pdf_path)
            all_recipes.extend(recipes)
    
    return all_recipes

# Specify the folder containing the PDFs
pdf_folder_path = r'C:\\Users\\oyku_\\Desktop\\Recipe Collector\\Recipe-Collector\\pdf_collectors\\test'

# Extract recipes from all PDFs in the folder
all_recipes = process_pdf_folder(pdf_folder_path)

# Check if any recipes were found before creating DataFrame
if all_recipes:
    recipes_df = pd.DataFrame(all_recipes)
    print("All Recipes DataFrame:\n", recipes_df)  # Print the DataFrame before saving
    recipes_df.to_csv('extracted_recipes3.csv', index=False, sep=';')
    print("Recipes extracted and saved to extracted_recipes.csv")
else:
    print("No recipes found in any PDFs.")


Extracted Text:
 TRADITIONAL
ITALIAN
RECIPES
COMENIUS PROJECT
”UNITED TASTES OF EUROPE”
201/2013
Istututo Comprensivo Statale di San Pietro Vernotico (BR)
ITALY
Classes 2D-2E-2F
Title Match: TRADITIONAL
Ingredients Match: None
Instructions Match: None
Missing one or more parts for this recipe.
Extracted Text:
 WINTER
SEASON
Title Match: WINTER
Ingredients Match: None
Instructions Match: None
Missing one or more parts for this recipe.
Extracted Text:
 CARTELLATE
INGREDIENTS
750 gr. flour
40 gr. olive oil
a pinch of salt
1 / 2 cup dry white wine
vincotto fig or grape or honey to taste
PREPARATION
Mix flour, oil and warm water, add salt and knead.
Roll out the dough and shape, cut with a special wheel larded, strips of 3 cm wide. Cut the
strips to 20 cm in length, and fold it in half, "tweezers" with your fingers from time to time
so as to form cavities, then roll it to form rosettes.
Allow to rest for about 4-5 hours to dry on a cloth.
In a saucepan heat the figs cooked wine or grape. Ex

In [37]:

import os
import re
import pdfplumber
import pandas as pd

def extract_recipes_from_pdf(pdf_path):
    recipes = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            print("Extracted Text:\n", text)  # Debugging line
            
            if text:
                # Extracting title (first line)
                title_pattern = r'^(.*)$'
                title_match = re.search(title_pattern, text, re.MULTILINE)
                title = title_match.group(1) if title_match else None

                # Extracting ingredients (all lines until the "Instructions" section)
                ingredients_pattern = r'(?i)ingredients\s*:\s*(.*?)(?=\n\s*instructions\s*:|$)'
                ingredients_match = re.search(ingredients_pattern, text, re.DOTALL)
                ingredients = ingredients_match.group(1).strip() if ingredients_match else None

                # Extracting instructions (all lines after the "Instructions" section)
                instructions_pattern = r'(?i)instructions\s*:\s*(.*?)(?=\n\s*$|$)'
                instructions_match = re.search(instructions_pattern, text, re.DOTALL)
                instructions = instructions_match.group(1).strip() if instructions_match else None

                # Print the extracted parts for debugging
                print("Title Match:", title)  # Debugging line
                print("Ingredients Match:", ingredients)  # Debugging line
                print("Instructions Match:", instructions)  # Debugging line

                # Append only if all parts are present
                if title and ingredients and instructions:
                    recipes.append({
                        'Title': title,
                        'Ingredients': ingredients,
                        'Instructions': instructions
                    })
                else:
                    print("Missing one or more parts for this recipe.")  # Log if any part is missing

    print("Total Recipes found:", len(recipes))  # Log the total count of recipes found
    return recipes

def process_pdf_folder(folder_path):
    all_recipes = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            recipes = extract_recipes_from_pdf(pdf_path)
            all_recipes.extend(recipes)
    
    return all_recipes

# Specify the folder containing the PDFs
pdf_folder_path = r'C:\Users\oyku_\Desktop\Recipe Collector\Recipe-Collector\pdf_collectors\test'

# Extract recipes from all PDFs in the folder
all_recipes = process_pdf_folder(pdf_folder_path)

# Specify the output CSV path
output_csv_path = r'C:\Users\oyku_\Desktop\Recipe Collector\Recipe-Collector\pdf_collectors\extracted_recipes.csv'

# Check if any recipes were found before creating DataFrame
if all_recipes:
    recipes_df = pd.DataFrame(all_recipes)
    print("All Recipes DataFrame:\n", recipes_df)  # Print the DataFrame before saving
    # Save the DataFrame to CSV
    recipes_df.to_csv(output_csv_path, index=False, sep=';')  # Saving to CSV with semicolon separator
    print(f"Recipes extracted and saved to {output_csv_path}")
else:
    print("No recipes found in any PDFs.")


Extracted Text:
 TRADITIONAL
ITALIAN
RECIPES
COMENIUS PROJECT
”UNITED TASTES OF EUROPE”
201/2013
Istututo Comprensivo Statale di San Pietro Vernotico (BR)
ITALY
Classes 2D-2E-2F
Title Match: TRADITIONAL
Ingredients Match: None
Instructions Match: None
Missing one or more parts for this recipe.
Extracted Text:
 WINTER
SEASON
Title Match: WINTER
Ingredients Match: None
Instructions Match: None
Missing one or more parts for this recipe.
Extracted Text:
 CARTELLATE
INGREDIENTS
750 gr. flour
40 gr. olive oil
a pinch of salt
1 / 2 cup dry white wine
vincotto fig or grape or honey to taste
PREPARATION
Mix flour, oil and warm water, add salt and knead.
Roll out the dough and shape, cut with a special wheel larded, strips of 3 cm wide. Cut the
strips to 20 cm in length, and fold it in half, "tweezers" with your fingers from time to time
so as to form cavities, then roll it to form rosettes.
Allow to rest for about 4-5 hours to dry on a cloth.
In a saucepan heat the figs cooked wine or grape. Ex